In [ ]:
dbutils.fs.mount(
     source= "wasbs://",
     mount_point = "/mnt/",
     extra_configs = {"fs.azure.account.key.storageaccount2024084j.blob.core.windows.net":""       
     }
)

In [ ]:
#silver 

dbutils.fs.mount(
     source= "wasbs://",
     mount_point = "/mnt/",
     extra_configs = {"fs.azure.account.key.storageaccount2024084j.blob.core.windows.net":""       
     }
)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:729)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:447)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:447)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [ ]:
#gold
dbutils.fs.mount(
     source= "wasbs://",
     mount_point = "/mnt/",
     extra_configs = {"fs.azure.account.key.storageaccount2024084j.blob.core.windows.net":""       
     }
)

In [ ]:
#criar database
spark.sql("CREATE DATABASE IF NOT EXISTS covid")

DataFrame[]

##  Camada Silver

In [ ]:
#ler CSV da camada bronze
df_covid_br_bronze = spark.read.format('csv').options(header='true', infer_schema='true',delimiter=';').load('dbfs:/mnt/storageaccount2024084j/bronze/SRAG_01-06.csv')

In [ ]:
#limpeza dados vazios 

df_covid_br_silver = df_covid_br_bronze.filter(df_covid_br_bronze.ID_REGIONA.isNotNull())

In [ ]:
#criar arquivo parquet no dir

df_covid_br_silver.write.format('delta').mode('overwrite').save('/mnt/storageaccount2024084j/silver/br_covid')

In [ ]:
dbutils.fs.mounts()


[MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/mnt/storageaccount2024084j/bronze', source='wasbs://bronze@storageaccount2024084j.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/storageaccount2024084j/gold', source='wasbs://gold@storageaccount2024084j.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/mnt/storageaccount2024084j/silver', source='wasbs://silver@storageaccount2024084j.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encr

In [ ]:
# Usar database
spark.sql('USE covid')

# Criar tabela
spark.sql("""
    CREATE TABLE IF NOT EXISTS br_covid
    USING DELTA
    LOCATION '/mnt/storageaccount2024084j/silver/br_covid'
    """)

In [ ]:
#query
br_covid= spark.sql('SELECT * FROM covid.br_covid')
display(br_covid)

## Camada Gold


In [ ]:
from pyspark.sql.functions import concat, to_date, year, month

In [ ]:
df_covid_br_silver = spark.read.format('delta').load('/mnt/storageaccount2024084j/silver/br_covid')

In [ ]:
# convert datas de string para date
df_covid_br_silver =  df_covid_br_silver.withColumn('DT_NOTIFIC', to_date(df_covid_br_silver['DT_NOTIFIC']))
df_covid_br_silver =  df_covid_br_silver.withColumn('DT_SIN_PRI', to_date(df_covid_br_silver['DT_SIN_PRI']))
df_covid_br_silver =  df_covid_br_silver.withColumn('DT_NASC', to_date(df_covid_br_silver['DT_NASC']))


In [ ]:
#adicionar nova coluna data da carga
from pyspark.sql.functions import current_date
df_covid_br_gold = df_covid_br_silver.withColumn('DT_CARGA',current_date())

In [ ]:
#renomear colunas
df_covid_br_gold = df_covid_br_gold.withColumnRenamed("DT_NOTIFIC","DT_NOTIFICACAO").withColumnsRenamed("DT_SIN_PRI","DT_PRIMEIROS_SINTOMAS").withColumnRenamed("DT_NASC","DT_NASCIMENTO")

In [ ]:
# adicionar a coluna 'year e 'month'
df_covid_br_gold = df_covid_br_gold.withColumn('YEAR', year(df_covid_br_gold['DT_NOTIFICACAO']))
df_covid_br_gold = df_covid_br_gold.withColumn('MONTH', month(df_covid_br_gold['DT_NOTIFICACAO']))

In [ ]:
#GRAVAR os dados de volta no Delta Lake, particionados por 'ano' e 'mes'
df_covid_br_gold.write.format('delta').mode('overwrite').option("mergeSchema","true").partitionBy('YEAR','MONTH').save('/mnt/storageaccount2024084j/gold')

In [ ]:
#use covid database
spark.sql("USE covid")

#cria uma tabela externa para o delta lake na camada gold

spark.sql("""
          DROP TABLE IF EXISTS br_covid_gold""")
spark.sql("""
          CREATE TABLE IF NOT EXISTS br_covid_gold
          USING DELTA
          LOCATION '/mnt/storageaccount2024084j/gold'
          """)

In [ ]:
#query
br_covid_gold = spark.sql("SELECT * FROM br_covid_gold")
display(br_covid_gold)


In [ ]:
#criar tabela agregada
from pyspark.sql.functions import count
slected_columns = ["year" "month","DT_NOTIFICACAO","SG_UF_NOT","ID_MUNICIP","DT_CARGA"]
df_select_columns_gold = df_covid_br_gold.select(slected_columns)
grouped_df = df_select_columns_gold.groupBy(slected_columns).agg(count("*").alias("count"))

In [ ]:
display(grouped_df)

In [ ]:
#gravar os dados de volta no delta lake particionados por 'ano' e 'mes'
grouped_df.write.format('delta').mode('overwrite').partitionBy('YEAR','MONTH').save('/mnt/storageaccount2024084j/gold/br_covid_gold_agg')